In [1]:
#importing the libraries
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from openpyxl import Workbook
import requests
import re
from time import sleep
import random

from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [ ]:
#setting the needed options
path = "C:\\Users\\eitan\\Downloads\\chromedriver_win32_new\\chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
driver = webdriver.Chrome(path, options=chrome_options)
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument('--no-sandbox')
chrome = webdriver.Chrome(path, options=chrome_options)

In [3]:
#logging in
url = f'https://www.etfrc.com/funds/stocklocator.php'
path = "C:\\Users\\eitan\\Downloads\\chromedriver_win32_new\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get(url)
user = "eitan.rosenfelder@mail.huji.ac.il"
psd = "eitan"
log = driver.find_element_by_class_name("pe-7s-unlock")
log.click()
sleep(random.randint(5, 7))
mail = driver.find_element_by_css_selector("#Email")
mail.send_keys(user)
password = driver.find_element_by_css_selector("#Password")
password.send_keys(psd)
sleep(random.randint(15, 17))
log_in = driver.find_element_by_xpath('/html/body/div[2]/section/div/div[3]/div/div/div[2]/form/button')
log_in.send_keys(Keys.RETURN)

In [ ]:
#a function making a data frame with etfs corresponding to the stock
def collection(stock):
    search = driver.find_element_by_xpath('//*[(@id = "stockTicker")]')
    search.send_keys(stock)
    enter = driver.find_element_by_css_selector("#locatorBtn")
    enter.send_keys(Keys.ENTER)
    sleep(random.randint(260, 265))
    how_many = driver.find_element_by_css_selector("#ETFTable_length .input-sm")
    how_many.send_keys("All")
    sleep(random.randint(33, 35))
    how_many.send_keys(Keys.ENTER)
    sleep(random.randint(300, 315))
    
    etf_list = []
    weight_list = []

    weight = driver.find_elements_by_css_selector("td:nth-child(3)")
    for i in weight:
        w = i.text
        w = w.replace("%","")
        w = w.replace(" ","")
        w = float(w)
        weight_list.append(w)
    etf = driver.find_elements_by_css_selector("td:nth-child(1)")
    for i in etf:
        etf_list.append(i.text)
    df = pd.DataFrame(list(zip(weight_list, etf_list)), 
                   columns =['weight', 'etf'])
    company = df[df.weight > 4]
    return company

In [ ]:
#calling the function for the stock list
stock_list = ["COP", "EOG", "BIIB", "HII", "AJRD", "INTC","F","CBOE", "AYI","AWI","OMC"]
for i in range(1,len(stock_list)):
    print(i)
    if i == 1:
        df_COP = collection(stock_list[i])
    if i == 2:
        df_EOG = collection(stock_list[i])
    if i == 3:
        df_BIIB = collection(stock_list[i])
    if i == 4:
        df_HII = collection(stock_list[i])
    if i == 5:
        df_AJRD = collection(stock_list[i])
    if i == 6:
        df_INTC = collection(stock_list[i])
    if i == 7:
        df_F = collection(stock_list[i])
    if i == 8:
        df_CBOE = collection(stock_list[i])
    if i == 9:
        df_AYI = collection(stock_list[i])
    if i == 10:
        df_AWI = collection(stock_list[i])
    if i == 11:
        df_OMC = collection(stock_list[i])

In [ ]:
#finding the best etf for CONOCOPHILLIPS
new_price=float(100)
chosen_etf = []
etf_list = df_COP["etf"].values.tolist()
for j in range(1,len(etf_list)):
    etf = etf_list[j]
    etf_price = driver.find_element_by_css_selector("#fundticker")
    etf_price.send_keys(etf)
    etf_price.send_keys(Keys.ENTER)
    sleep(random.randint(27, 30))
    price = driver.find_element_by_css_selector(".row:nth-child(1) tr:nth-child(7) td+ td").text
    print(price)
    price = price.replace("bp","")
    price = price.replace(" ","")
    if price.isnumeric() == True:
        print(price)
        price = float(price)
        if price <= new_price:
            if price > 0:
                new_price = price
                chosen_etf.append(etf)
    else: 
        continue
chosen_etf

In [ ]:
#result
best_etf = chosen_etf
price_COP = new_price
best_etf

In [ ]:
#the best etfs for the companies
def best(df):
    new_price=float(100)
    chosen_etf = []
    etf_list = df["etf"].values.tolist()
    for j in range(1,len(etf_list)):
        etf = etf_list[j]
        etf_price = driver.find_element_by_css_selector("#fundticker")
        etf_price.send_keys(etf)
        etf_price.send_keys(Keys.ENTER)
        sleep(random.randint(27, 30))
        price = driver.find_element_by_css_selector(".row:nth-child(1) tr:nth-child(7) td+ td").text
        price = price.replace("bp","")
        price = price.replace(" ","")
        if price.isnumeric() == True:
            price = float(price)
            if price <= new_price:
                if price > 0:
                    new_price = price
                    chosen_etf.append(etf)
        else: 
            continue
    return new_price, chosen_etf


In [ ]:
#calling the function
price_EOG,best_EOG = best(df_EOG)
print('EO')
price_BIIB,best_BIIB = best(df_BIIB)
print('BI')
price_HII,best_HII = best(df_HII)
print('HI')
price_AJRD,best_AJRD = best(df_AJRD)
print('aj')
price_INTC,best_INTC = best(df_INTC)
print('in')
price_F,best_F = best(df_F)
price_CBOE,best_CBOE = best(df_CBOE)
print('CB')
price_AYI,best_AYI = best(df_AYI)
print('aY')
price_AWI,best_AWI = best(df_AWI)
print('AW')
price_OMC,best_OMC = best(df_OMC)
print('OM')


In [ ]:
#making a full list without duplicates
best_set = best_etf
def common(best_set,new_set):
    for value in new_set:
        if value in best_set:
            return best_set
    best_set.append(new_set)
    return best_set
best_set = common(best_set,best_EOG)
best_set = common(best_set,best_BIIB)
best_set = common(best_set,best_HII)
best_set = common(best_set,best_AJRD)
best_set = common(best_set,best_INTC)
best_set = common(best_set,best_CBOE)
best_set = common(best_set,best_AYI)
best_set = common(best_set,best_AWI)
best_set = common(best_set,best_OMC)
best_set

In [ ]:
#preparing the final selection of etfs
def split_two_parts(full_list, n):
    return n_list[:n], n_list[n:]
best_set = split_two_parts(best_set, len(best_etf))
def extracting_best(lst): 
    return [item[0] for item in lst] 
best_etfs = extracting_best(best_set)
best_etfs